<a href="https://colab.research.google.com/github/GabrielaCuervoR/Analitica3/blob/main/funciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer ### para imputación
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
import joblib
from sklearn.preprocessing import StandardScaler ## escalar variables

In [17]:
###########Esta función permite ejecutar un archivo  con extensión .sql que contenga varias consultas
def ejecutar_sql (nombre_archivo, cur):
  sql_file=open(nombre_archivo)
  sql_as_string=sql_file.read()
  sql_file.close
  cur.executescript(sql_as_string)

In [18]:
#Función para imputar las variables categoricas
def imputar_fc(df, list_cat):
  df_c = df[list_cat]

  imputer_c = SimpleImputer(strategy='most_frequent')
  imputer_c.fit(df_c)
  X_c = imputer_c.transform(df_c)
  df_c = pd.DataFrame(X_c, columns=df_c.columns)

  df[list_cat] = df_c  # Actualizar las columnas imputadas en el DataFrame original
  return df

In [19]:
#Función para imputar variables numericas:
def imputar_fn(df, list_num):
  df_n = df[list_num]

  imputer_n = SimpleImputer(strategy='median')
  imputer_n.fit(df_n)
  X_n = imputer_n.transform(df_n)
  df_n = pd.DataFrame(X_n, columns=df_n.columns)

  df[list_num] = df_n  # Actualizar las columnas imputadas en el DataFrame original
  return df

In [20]:
##Función para seleccionar variables
def sel_variables(modelos,X,y,threshold):

  var_names_ac=np.array([])
  for modelo in modelos:
      #modelo=modelos[i]
      modelo.fit(X,y)
      sel = SelectFromModel(modelo, prefit=True,threshold=threshold)
      var_names= modelo.feature_names_in_[sel.get_support()]
      var_names_ac=np.append(var_names_ac, var_names)
      var_names_ac=np.unique(var_names_ac)

  return var_names_ac

In [21]:
##Función para los modelos
def medir_modelos(modelos,scoring,X,y,cv):

  metric_modelos=pd.DataFrame()
  for modelo in modelos:
      scores=cross_val_score(modelo,X,y, scoring=scoring, cv=cv )
      pdscores=pd.DataFrame(scores)
      metric_modelos=pd.concat([metric_modelos,pdscores],axis=1)

  metric_modelos.columns=["logistic_regresion","random_forest","decision_tree","gradient_boosting"]
  return metric_modelos